<h1>Preparation of the German Twitter Data</h1>

<p>This file is for preparing the german tweets. The preparation is mainly for training my own model for sentiment detection with Flair. </p>

In [1]:
import csv
import numpy as np
import pandas as pd
import re
import nltk

<h1>Einlesen und bestimmung der Felder</h1>
<p>Durch die weitern Operatoren, kann ich die einzelnen Felder in DF bestimmen. Durch die Lambda Funktion werden die Str zu Listen, was die spätere verarbeitung erleichtert. 
    Durch die Parse_dates Funktion wird das Datumsfeld zu einem Timestamp, was die weiter Verarbeitung wesentlich leichter macht.</p>

In [30]:
df_manager_de = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Manager_DE/data.csv", sep=";", 
                            converters={"tweet_type": lambda x: x.strip("[]").replace("'", "").split(", "),
                                        "referenced_tweet_id": lambda x: x.strip("[]").replace("'", "").split(", ")},
                           parse_dates=["created_at"])

In [31]:
df_manager_de.head()

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text
0,703311366262800384,1392131804,2016-02-26 20:11:11+00:00,703311366262800384,de,0,0,2,0,Twitter Web Client,[None],[None],#Wirecard-Chef Markus Braun pumpt irrwitzige S...
1,702405572012974080,1053723204,2016-02-24 08:11:53+00:00,702405572012974080,de,0,0,0,0,Twitter Web Client,[None],[None],Jetzt günstig einsammeln und später absahnen? ...
2,701784983829143553,554491983,2016-02-22 15:05:53+00:00,701784983829143553,de,0,0,0,0,Facebook,[None],[None],Morgen startet die neue Reihe R²-JazzCube! Vol...
3,701446667170615296,3365669884,2016-02-21 16:41:32+00:00,701446667170615296,de,0,0,1,0,Twitter Web Client,[None],[None],die Y-Achse des #prusai3 nimmt Formen an :-) ...
4,700597812556386304,4858383167,2016-02-19 08:28:29+00:00,700597812556386304,de,2,0,0,0,Twitter for iPhone,[retweeted],[700578193334411264],RT @aktionaer: #Wirecard: Was führt der Vorsta...


<p>Prüfen welche Datenfelder in den einzelnen Zeilen enthalten sind. War wichtig um diese anpassen zu können.</p>

In [32]:
print("conversation_id: ", df_manager_de.iloc[0]["conversation_id"], "type: ", type(df_manager_de.iloc[0]["conversation_id"]))
print("author_id: ", df_manager_de.iloc[0]["author_id"], "type: ", type(df_manager_de.iloc[0]["author_id"]))
print("created_at: ", df_manager_de.iloc[0]["created_at"], "type: ", type(df_manager_de.iloc[0]["created_at"]))
print("tweet_id: ", df_manager_de.iloc[0]["tweet_id"], "type: ", type(df_manager_de.iloc[0]["tweet_id"]))
print("lang: ", df_manager_de.iloc[0]["lang"], "type: ", type(df_manager_de.iloc[0]["lang"]))
print("retweet_count: ", df_manager_de.iloc[0]["retweet_count"], "type: ", type(df_manager_de.iloc[0]["retweet_count"]))
print("reply_count: ", df_manager_de.iloc[0]["reply_count"], "type: ", type(df_manager_de.iloc[0]["reply_count"]))
print("like_count: ", df_manager_de.iloc[0]["like_count"], "type: ", type(df_manager_de.iloc[0]["like_count"]))
print("quote_count: ", df_manager_de.iloc[0]["quote_count"], "type: ", type(df_manager_de.iloc[0]["quote_count"]))
print("source: ", df_manager_de.iloc[0]["source"], "type: ", type(df_manager_de.iloc[0]["source"]))
print("tweet_type: ", df_manager_de.iloc[0]["tweet_type"], "type: ", type(df_manager_de.iloc[0]["tweet_type"]))
print("referenced_tweet_id: ", df_manager_de.iloc[0]["referenced_tweet_id"], "type: ", type(df_manager_de.iloc[0]["referenced_tweet_id"]))
print("text: ", df_manager_de.iloc[0]["text"], "type: ", type(df_manager_de.iloc[0]["text"]))

conversation_id:  703311366262800384 type:  <class 'numpy.int64'>
author_id:  1392131804 type:  <class 'numpy.int64'>
created_at:  2016-02-26 20:11:11+00:00 type:  <class 'pandas._libs.tslibs.timestamps.Timestamp'>
tweet_id:  703311366262800384 type:  <class 'numpy.int64'>
lang:  de type:  <class 'str'>
retweet_count:  0 type:  <class 'numpy.int64'>
reply_count:  0 type:  <class 'numpy.int64'>
like_count:  2 type:  <class 'numpy.int64'>
quote_count:  0 type:  <class 'numpy.int64'>
source:  Twitter Web Client type:  <class 'str'>
tweet_type:  ['None'] type:  <class 'list'>
referenced_tweet_id:  ['None'] type:  <class 'list'>
text:  #Wirecard-Chef Markus Braun pumpt irrwitzige Summen in sein Unternehmen. Woher stammt das Geld? @SchreiberDohms  https://t.co/VdsV4nvBe4 type:  <class 'str'>


In [17]:
print("tweet_type: ", df_manager_de.iloc[4]["tweet_type"][0], "type: ", type(df_manager_de.iloc[4]["tweet_type"][0]))

tweet_type:  retweeted type:  <class 'str'>


In [18]:
print("tweet_type: ", df_manager_de.iloc[4]["tweet_type"], "type: ", type(df_manager_de.iloc[4]["tweet_type"]))

tweet_type:  ['retweeted'] type:  <class 'list'>


In [40]:
df_manager_de.iloc[0]["created_at"]

Timestamp('2016-02-26 20:11:11+0000', tz='UTC')

In [43]:
df_manager_de.iloc[0]["created_at"].dayofweek

4

In [11]:
source_list = list(df_manager_de["source"])

In [44]:
print("text: ", df_manager_de.iloc[0]["text"], "type: ", type(df_manager_de.iloc[0]["text"]))

text:  #Wirecard-Chef Markus Braun pumpt irrwitzige Summen in sein Unternehmen. Woher stammt das Geld? @SchreiberDohms  https://t.co/VdsV4nvBe4 type:  <class 'str'>


In [45]:
if "Braun" in df_manager_de.iloc[0]["text"]:
    print("Jap ist da")

Jap ist da


<p>So kann ich prüfen ob bestimmte Begriffe in den Texten enthalten sind und mir die conversation_id ausgeben lassen.</p>

In [50]:
for each in range(len(df_manager_de)):
    if "BVB" in df_manager_de.iloc[each]["text"]:
        print(df_manager_de.iloc[each]["conversation_id"])

722861750723284993
722858374807822336
739490204180992000
798533984854544384
798522085756678144
798516440277995520
926889966369689600
984792434533191680
984778759629885440
1088115106519633920
1260905841433198592
1260900850077179905
1260881271372288000
1260880569224900610
1278590734430547969
1278581125951225858
1278578905042034688
1278569106493640704
1278564053112295425
1278443988769542151
1278436721462513666
1278435775441494016
1278435182056542212
1278434921594454019
1278434819685441537
1278434701418651650


In [23]:
source_unique_list = list(np.unique(source_list))

In [27]:
df_user_manager_de = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Manager_DE/user.csv", sep=";")

In [28]:
df_user_manager_de.head()

,user_id,username,name,follower,following,tweet_cound,list_count,verified,created_at
0,1392131804,capitalMagazin,Capital,7669,436,6287,212,False,2013-04-30 13:55:36+00:00
1,1053723204,l_m_s15,Lukas Spang,2292,237,39499,92,False,2013-01-01 23:31:19+00:00
2,554491983,bvb_remmel,bvb.remmel,70,66,1831,0,False,2012-04-15 15:05:15+00:00
3,3365669884,MarkusB1980,Markus Braun,82,181,620,13,False,2015-07-08 08:43:24+00:00
4,4858383167,QuasselDax,PrivateTraderContact,8,17,61,4,False,2016-01-28 20:15:04+00:00


In [35]:
df_user_manager_de["name"].where(df_user_manager_de["name"] == "Markus Braun")

0                 NaN
1                 NaN
2                 NaN
3        Markus Braun
4                 NaN
             ...     
10407             NaN
10408             NaN
10409             NaN
10410             NaN
10411             NaN
Name: name, Length: 10412, dtype: object

In [54]:
df_user_manager_de[df_user_manager_de["name"].values == "Markus"]

,user_id,username,name,follower,following,tweet_cound,list_count,verified,created_at
5980,348086065,Leineems,Markus,19,156,248,1,False,2011-08-03 21:16:41+00:00


In [48]:
df_user_manager_de.iloc[18]

user_id                       3365669884
username                     MarkusB1980
name                        Markus Braun
follower                              82
following                            181
tweet_cound                          620
list_count                            13
verified                           False
created_at     2015-07-08 08:43:24+00:00
Name: 18, dtype: object